<a href="https://colab.research.google.com/github/alirezasdg1/Amazon-recommender-system/blob/master/notebooks/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocess the datasets

First I downlaod the dataset and prepare them for furthure analyses.


In [14]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from google.colab import files
import numpy as np

Dataset for the products

In [3]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Sports_and_Outdoors.json.gz

--2020-09-14 17:10:05--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Sports_and_Outdoors.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512767929 (489M) [application/octet-stream]
Saving to: ‘meta_Sports_and_Outdoors.json.gz’

meta_Sports_and_Out 100%[===================>] 489.01M  55.4MB/s    in 9.1s    

2020-09-14 17:10:14 (53.7 MB/s) - ‘meta_Sports_and_Outdoors.json.gz’ saved [512767929/512767929]



In [4]:
### load the meta data

data = []
with gzip.open("meta_Sports_and_Outdoors.json.gz") as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

962300
{'category': ['Sports & Outdoors', 'Sports & Fitness', 'Other Sports', 'Dance', 'Clothing'], 'tech1': '', 'description': ['3 layers of super-soft polyester tulle can be cut to length. Satin covered stretch elastic waist gives a one size fits all most.'], 'fit': '', 'title': 'Adult Tutu Assorted Colors (Turquoise)', 'also_buy': [], 'image': [], 'tech2': '', 'brand': 'BubuBibi', 'feature': ['3 Layers - 100% Polyester Tulle, Hand Wash Lay FLat to Dry', '100% Polyester', 'FIT: most Kids. Teens . Adults *Please Check Measurements for Proper Fit', 'WAIST: Satin Covered Elastic 18" relaxed to 36" fully stretched', 'LENGTH: 16" to 17" *can easily be cut to desired length without worry of fraying or unraveling', 'You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.', '<a id="creturns-policy-url-text-pc" class="a-link-normal" href="/b/ref=frt_dp_sm?ie=UTF8&node=8622296011">\n                        Read 

In [5]:
# convert list into pandas dataframe

df_products = pd.DataFrame.from_dict(data)

print(len(df_products))

962300


# Product dataframe

In [6]:
df_products.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

In [7]:
df_products.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Adult Tutu Assorted Colors (Turquoise),[],[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (","[B071LQWQBQ, B00M14DG0O, B076GTTY9W, B07BSZ4GY...","<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,NaN
1,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Bububibi Adult Ballet Tutu Cheetah Pink,[],[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,NaN
2,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Girls Ballet Tutu Neon Orange,[B01MYHD3LV],[],,Unknown,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#924,198 in Toys & Games (See Top 100 in Toy...","[B0152HBC52, B07C29WWV1, B01M0BIP9H, B00VXJHFR...",Toys & Games,,,$5.70,0000031860,NaN
3,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,0000031852,NaN
4,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[Dance tutu for girls ages 2-8 years. Perfect ...,,Girls Ballet Tutu Neon Blue,"[B009THXHPA, B00BEU1ZBI, B079PYLZTM, B00BEU1ZR...",[],,Bububibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#393,597 in Toys & Games (See Top 100 in Toy...","[B009THXHPA, B079PYLZTM, B079PQPLT3, B00UP6A3R...",Toys & Games,,,$6.54,0000031895,NaN


In [13]:
df_products.to_csv('df_products.csv')

In [15]:
files.download("df_products.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
df_products[df_products.astype(str)['image'] != '[]']

(476483, 18)

## Review data

In [17]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Sports_and_Outdoors.json.gz

--2020-09-14 17:54:01--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Sports_and_Outdoors.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1690448337 (1.6G) [application/octet-stream]
Saving to: ‘Sports_and_Outdoors.json.gz’

Sports_and_Outdoors 100%[===================>]   1.57G  49.7MB/s    in 33s     

2020-09-14 17:54:35 (48.9 MB/s) - ‘Sports_and_Outdoors.json.gz’ saved [1690448337/1690448337]



In [18]:
data = []
with gzip.open('Sports_and_Outdoors.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

12980837
{'overall': 5.0, 'verified': False, 'reviewTime': '02 1, 2014', 'reviewerID': 'A23K73OVXJ04EG', 'asin': '0000031895', 'reviewerName': 'Lamb612000', 'reviewText': 'It was as described and fit my 6 year old grand daughter perfectly. She loves to play dress-up and this is a good addition to her wardrobe. I received the item quickly and the packaging was great.', 'summary': 'Neon Blue Tutu', 'unixReviewTime': 1391212800}


In [19]:
df_review = pd.DataFrame.from_dict(data)

print(len(df_review))

12980837


In [20]:
del data

In [ ]:
df_review.head()

In [23]:
df_review.to_csv('df_review.csv')

In [24]:
files.download("df_review.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save dataframe to google drive 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_product = pd.read_csv("/content/drive/My Drive/Datasets/df_products.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_product.shape

(962300, 19)

In [6]:
df_review = pd.read_csv("/content/drive/My Drive/Datasets/df_review.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_review.shape

(12980837, 13)

In [21]:
!cp /content/df_products.csv "/content/My Drive/Datasets"

cp: cannot stat '/content/df_products.csv': No such file or directory


In [22]:
!cp /content/df_review.csv "/content/My Drive/Datasets"

cp: cannot stat '/content/df_review.csv': No such file or directory


In [7]:
df_product.head(2)

,Unnamed: 0,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,0,"['Sports & Outdoors', 'Sports & Fitness', 'Oth...",NaN,['3 layers of super-soft polyester tulle can b...,NaN,Adult Tutu Assorted Colors (Turquoise),[],[],NaN,BubuBibi,"['3 Layers - 100% Polyester Tulle, Hand Wash L...","712,899 in Clothing, Shoes & Jewelry (","['B071LQWQBQ', 'B00M14DG0O', 'B076GTTY9W', 'B0...","<img src=""https://images-na.ssl-images-amazon....",NaN,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,NaN
1,1,"['Sports & Outdoors', 'Sports & Fitness', 'Oth...",NaN,['3 layers of super-soft polyester tulle can b...,NaN,Bububibi Adult Ballet Tutu Cheetah Pink,[],[],NaN,BubuBibi,"['3 Layers - 100% Polyester Tulle, Hand Wash L...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",NaN,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,NaN


In [8]:
df_review.head(2)

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,0,5.0,False,"02 1, 2014",A23K73OVXJ04EG,0000031895,Lamb612000,It was as described and fit my 6 year old gran...,Neon Blue Tutu,1391212800,NaN,NaN,NaN
1,1,4.0,True,"11 20, 2013",A2681T699HV6H1,0000031895,Courtney R,very cute and went great with my Halloween cos...,loved it!,1384905600,NaN,NaN,NaN


## Remove products without images

In [9]:
df_product_img = df_product[df_product.astype(str)['image'] != '[]'].copy()

In [10]:
df_product_img.shape

(476483, 19)

In [11]:
df_product_img.head(2)

,Unnamed: 0,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
6,6,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",NaN,['Deluxe Duffel Bag with full length zipper de...,NaN,6ft of Acid Free Linen Tape for Bookbinding,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,Stansport,"['Cotton Canvas', 'Acid Free']","2,077,567 in Sports & Outdoors (","['B003W5YN9Y', 'B077FXJTTS']",Sports & Outdoors,NaN,NaN,$4.00,0000012300,NaN
11,11,"['Sports & Outdoors', 'Sports & Fitness', 'Gol...",NaN,['Create an instant indoors putting green by p...,NaN,Putting Set in Case,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,Allegra,[],"5,302,408 in Sports & Outdoors (",[],Sports & Outdoors,NaN,NaN,NaN,0500615594,NaN


In [12]:
df_all = pd.merge(df_product_img,df_review,on='asin')

In [53]:
df_all.head(1)

,Unnamed: 0_x,category,tech1,description,fit,title,also_buy,image_x,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details,Unnamed: 0_y,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,unixReviewTime,vote,style,image_y
0,6,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",NaN,['Deluxe Duffel Bag with full length zipper de...,NaN,6ft of Acid Free Linen Tape for Bookbinding,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,Stansport,"['Cotton Canvas', 'Acid Free']","2,077,567 in Sports & Outdoors (","['B003W5YN9Y', 'B077FXJTTS']",Sports & Outdoors,NaN,NaN,$4.00,0000012300,NaN,9411978,4.0,True,"09 7, 2017",A1BUNBDDEO1N4B,Tom,not used yet but looks fine,Four Stars,1504742400,NaN,NaN,NaN


In [13]:
df_all.columns

Index(['Unnamed: 0_x', 'category', 'tech1', 'description', 'fit', 'title',
       'also_buy', 'image_x', 'tech2', 'brand', 'feature', 'rank', 'also_view',
       'main_cat', 'similar_item', 'date', 'price', 'asin', 'details',
       'Unnamed: 0_y', 'overall', 'verified', 'reviewTime', 'reviewerID',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image_y'],
      dtype='object')

In [70]:
df_test = df_all.groupby('reviewerID').agg(Total_review_C=('asin', 'count')).reset_index()

In [72]:
df_test2 = df_all.groupby('asin').agg(Total_review_P=('reviewerID', 'count')).reset_index()

In [73]:
df_test

,reviewerID,Total_review_C
0,A0000040I1OM9N4SGBD8,1
1,A0000074RA15UCBH3ON5,1
2,A000013090ZI3HIT9N5V,3
3,A0000196KBA0ICH151EG,1
4,A00008882A0PUVHCTDUP,2
...,...,...
5554032,AZZZV0D9D5V05,1
5554033,AZZZVAXZQB7JJ,7
5554034,AZZZWV7EIR8PG,1
5554035,AZZZY1W55XHZR,3


In [79]:
df_test2

,asin,Total_review_P
0,0000012300,1
1,0500615594,1
2,0692747222,14
3,0899332757,94
4,0899333257,160
...,...,...
474965,B01HJGDIBK,4
474966,B01HJGDVEE,1
474967,B01HJGIZ2W,11
474968,B01HJGNPLI,1


In [76]:
df_new = pd.merge(df_all,df_test,on='reviewerID', how='left')

In [81]:
df_new.head(1)

,Unnamed: 0_x,category,tech1,description,fit,title,also_buy,image_x,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details,Unnamed: 0_y,overall,verified,reviewTime,reviewerID,reviewerName,reviewText,summary,unixReviewTime,vote,style,image_y,product_count,Total_review_C,Total_review_P
0,6,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",NaN,['Deluxe Duffel Bag with full length zipper de...,NaN,6ft of Acid Free Linen Tape for Bookbinding,[],['https://images-na.ssl-images-amazon.com/imag...,NaN,Stansport,"['Cotton Canvas', 'Acid Free']","2,077,567 in Sports & Outdoors (","['B003W5YN9Y', 'B077FXJTTS']",Sports & Outdoors,NaN,NaN,$4.00,0000012300,NaN,9411978,4.0,True,"09 7, 2017",A1BUNBDDEO1N4B,Tom,not used yet but looks fine,Four Stars,1504742400,NaN,NaN,NaN,NaN,1,1


In [89]:
df_new[(df_new['Total_review_P']>15) & (df_new['Total_review_C']>15)].shape

(370260, 34)

In [66]:
df_new[df_new["Total_review"] >= 10].shape

(917559, 33)

In [69]:
df_new2[df_new2["Total_review"] >= 10].shape

(8832112, 33)

In [59]:
df_all[df_all["product_count"] >= 10].sum()

Unnamed: 0_x      0.0
category          0.0
tech1             0.0
description       0.0
fit               0.0
title             0.0
also_buy          0.0
image_x           0.0
tech2             0.0
brand             0.0
feature           0.0
rank              0.0
also_view         0.0
main_cat          0.0
similar_item      0.0
date              0.0
price             0.0
asin              0.0
details           0.0
Unnamed: 0_y      0.0
overall           0.0
verified          0.0
reviewTime        0.0
reviewerID        0.0
reviewerName      0.0
reviewText        0.0
summary           0.0
unixReviewTime    0.0
vote              0.0
style             0.0
image_y           0.0
product_count     0.0
dtype: float64

In [29]:
df_test[(df_test.Total_review > 10)].shape

(103294, 2)

In [ ]:
df_all[df_test.asin]

In [21]:
byasin = df_all.groupby('asin').aggregate('count')
tags = byasin[byasin.reviewerID >= 10].index
data[data['asin'].isin(tags)].shape

NameError: ignored

In [15]:
df_all.drop(['tech1', 'tech2','Unnamed: 0_x','fit','similar_item','date','details','Unnamed: 0_y','reviewTime','reviewerID','reviewerName','vote','image_y'], axis=1,inplace=True)

In [16]:
df_all.head()

,category,description,title,also_buy,image_x,brand,feature,rank,also_view,main_cat,price,asin,overall,verified,reviewText,summary,unixReviewTime,style
0,"['Sports & Outdoors', 'Outdoor Recreation', 'C...",['Deluxe Duffel Bag with full length zipper de...,6ft of Acid Free Linen Tape for Bookbinding,[],['https://images-na.ssl-images-amazon.com/imag...,Stansport,"['Cotton Canvas', 'Acid Free']","2,077,567 in Sports & Outdoors (","['B003W5YN9Y', 'B077FXJTTS']",Sports & Outdoors,$4.00,0000012300,4.0,True,not used yet but looks fine,Four Stars,1504742400,NaN
1,"['Sports & Outdoors', 'Sports & Fitness', 'Gol...",['Create an instant indoors putting green by p...,Putting Set in Case,[],['https://images-na.ssl-images-amazon.com/imag...,Allegra,[],"5,302,408 in Sports & Outdoors (",[],Sports & Outdoors,NaN,0500615594,2.0,True,Bent golf stick,DR. BINO,1435449600,NaN
2,"['Sports & Outdoors', 'Sports & Fitness', 'Exe...","[""Apollo Challenge is the most effective and s...",Apollo Challenge : The 28-Day System To Look A...,"['B0713Z5DGM', 'B01KUDOLN8', 'B01DJBCN3I', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,Apollo Challenge,"['Reduce Belly Fat, Increase Muscle Definition...","288,369 in Sports & Outdoors (",[],Sports & Outdoors,$9.97,0692747222,5.0,False,Simple and Powerful! So much non-sense out the...,Simple and easy to follow - it gives actual re...,1477612800,{'Format:': ' Perfect Paperback'}
3,"['Sports & Outdoors', 'Sports & Fitness', 'Exe...","[""Apollo Challenge is the most effective and s...",Apollo Challenge : The 28-Day System To Look A...,"['B0713Z5DGM', 'B01KUDOLN8', 'B01DJBCN3I', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,Apollo Challenge,"['Reduce Belly Fat, Increase Muscle Definition...","288,369 in Sports & Outdoors (",[],Sports & Outdoors,$9.97,0692747222,5.0,False,This program is incredible. I did this with m...,A Plan that actually work!.... If you actually...,1477526400,{'Format:': ' Perfect Paperback'}
4,"['Sports & Outdoors', 'Sports & Fitness', 'Exe...","[""Apollo Challenge is the most effective and s...",Apollo Challenge : The 28-Day System To Look A...,"['B0713Z5DGM', 'B01KUDOLN8', 'B01DJBCN3I', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,Apollo Challenge,"['Reduce Belly Fat, Increase Muscle Definition...","288,369 in Sports & Outdoors (",[],Sports & Outdoors,$9.97,0692747222,5.0,False,The Apollo Challange is what you've been looki...,The Apollo Challenge just plain works...,1477526400,{'Format:': ' Perfect Paperback'}


In [17]:
del df_product_img, df_review

## NLP text

In [42]:
import re
import urllib
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
def text_process(text):
        text = re.sub(r'<[^>]*>', ' ', text).strip()
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = re.sub('[‘’“”…]', '', text)
        text = re.sub('\n', '', text)
        nopunc = [char for char in text if char not in string.punctuation]

        nopunc = ''.join(nopunc)
        list_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
     
        stemmer_snowball = PorterStemmer()
 
        tokens_stemsnowball = [stemmer_snowball.stem(sent) for sent in list_words]

        return ' '.join(tokens_stemsnowball)


In [19]:
def nlp_process(df):

    df["text"] = df["description"]+df["brand"]+df["feature"]+df["reviewText"]+df["summary"]+df['style']
        

    df['text'] = df['text'].apply(text_process)

    # vectorizer = TfidfVectorizer(min_df = 3,max_df = 0.8, max_features = 1200)
    # m_nlp = vectorizer.fit_transform(df["text"]).toarray()
    # df_nlp = pd.DataFrame(m_nlp, columns=vectorizer.get_feature_names())
    
    return df_nlp

In [ ]:
df_all = nlp_process(df_all)

In [73]:
df_all["style"].iloc[9794153]

"{'Size:': ' Small', 'Color:': ' Black/White'}"